Цель данного домашнего задания — познакомить вас с концепцией и реализацией RAG в LLM, а также развить навыки интеграции механизмов поиска с языковыми моделями. Вы научитесь извлекать данные из специализированных источников, использовать их для поддержки генеративного процесса и оценивать качество полученных результатов.

# Устанавливаем зависимости

In [1]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ixuiyzlp
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-ixuiyzlp
  Resolved https://github.com/huggingface/transformers.git to commit d5f992f5e67e8ced64a90fe01dfc50796fd2933d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.53.0.dev0-py3-none-any.whl size=11374517 sha256=24511088781bfa7e5674df6e5e5a1e18691efe8c330e83bba68ce1804742fa84
  Stored in directory: /tmp/pip-ephem-wheel-cache-mahdlubt/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [2]:
!pip install datasets sentence_transformers==3.3.1 trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Текст на чанки - 5 баллов

Для наших заданий будем использовать датасет rag-dataset-12000, в котором есть вопросы, ответы и большие контексты.

In [3]:
from datasets import load_dataset
wikiq = load_dataset('neural-bridge/rag-dataset-12000')

print(len(wikiq['train']))
print(wikiq['train'][0]['question'], '\n')
print(wikiq['train'][0]['answer'], '\n')
print(wikiq['train'][0]['context'], '\n')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

(…)-00000-of-00001-9df3a936e1f63191.parquet:   0%|          | 0.00/23.1M [00:00<?, ?B/s]

(…)-00000-of-00001-af2a9f454ad1b8a3.parquet:   0%|          | 0.00/5.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2400 [00:00<?, ? examples/s]

9600
What is the Berry Export Summary 2028 and what is its purpose? 

The Berry Export Summary 2028 is a dedicated export plan for the Australian strawberry, raspberry, and blackberry industries. It maps the sectors’ current position, where they want to be, high-opportunity markets, and next steps. The purpose of this plan is to grow their global presence over the next 10 years. 

Caption: Tasmanian berry grower Nic Hansen showing Macau chef Antimo Merone around his property as part of export engagement activities.
THE RISE and rise of the Australian strawberry, raspberry and blackberry industries has seen the sectors redouble their international trade focus, with the release of a dedicated export plan to grow their global presence over the next 10 years.
Driven by significant grower input, the Berry Export Summary 2028 maps the sectors’ current position, where they want to be, high-opportunity markets and next steps.
Hort Innovation trade manager Jenny Van de Meeberg said the value an

Для налаживания процессов возьме 100 самых длинных текстов

In [4]:
k_longest = 100
train_data = sorted(wikiq['train'], key=lambda w: len(w["context"]))[-k_longest:]

train_full_docs = [elem['context'] for elem in train_data]
train_queries = [elem['question'] for elem in train_data]

Реализуйте класс Chunker, в котором метод split_text_to_chunks разбивает входный текст на чанки с перекрытием и возвращает список полученных чанков, и метод get_chunked_list, которые по списку текстов возвращает список чанков из этих текстов согласно методу split_text_to_chunks. Размер чанка и перекрытие измеряется в кол-ве слов, то есть chunk_words = 10 означает, что чанк состоит из 10 слов (слова - сущности, которые получаются после простого сплита строки по одному пробелу " "), перекрытие 3 означает, что если взять два соседний чанка, то 3 последних слова первого являются 3мя первыми словами второго чанка.

In [5]:
class Chunker:
    def __init__(self, chunk_words: int = 100, overlap: int = 30):
        # your code here
        self.chunk_words = chunk_words
        self.overlap = overlap

    def split_text_to_chunks(self, text: str) -> list[str]:
        """Метод разбивает входный текст на чанки с перекрытием и возвращает список полученных чанков."""
        sep_words = text.split(" ")

        l, r = 0, self.chunk_words
        chunks = []
        while l <= len(sep_words):
            chunks.append(" ".join(sep_words[l:r]))
            l = r - self.overlap
            r = l + self.chunk_words
        return chunks


    def get_chunked_list(self, texts: list[str]) -> list[str]:
        """Метод по списку текстов возвращает список чанков из этих текстов."""
        all_chunks = []
        for txt in texts:
            all_chunks += self.split_text_to_chunks(text=txt)
        return all_chunks

In [6]:
chunker = Chunker(chunk_words=100, overlap=30)
train_chunked_docs = chunker.get_chunked_list(train_full_docs)
print(train_chunked_docs, '\n')
print(train_chunked_docs[0], '\n')
print(train_chunked_docs[1])

['The path to caregiver can be long and slow as one watches a friend or loved one slowly show signs of decline. It can also be quick and unexpected, brought on by sudden illness or accident. Slow or quick, this is the most difficult and often turbulent time for everyone involved. The first hurdle is recognizing and/or admitting that your loved one needs help. At the same time it is most difficult for the patient to admit the need for help.\nThere can be a tremendous amount of denial as we begin to see the signs. At first this is usually', 'most difficult for the patient to admit the need for help.\nThere can be a tremendous amount of denial as we begin to see the signs. At first this is usually facilitated by the loved one\'s struggle to hide changes in ability. Vanity, pride, and stubbornness are very strong motivators and during this period they may intensify. The patient can be very clever at hiding difficulties. Often the future caregiver is only too eager to believe that everythin

In [7]:
print(len(train_chunked_docs))

1639


# Векторный поиск - 10 баллов

Построить простой векторный поиск на основе энкодерной модели modernbert-embed-base. Модель устроена так, что эмбеддинги запросов (всегда должны начинаться с префикса "search_query: ") всегда близки в векторном пространстве эмбеддингам похожих/релевантных документов (их текст всегда должен начинаться на "search_document: ")

In [8]:
from sentence_transformers import SentenceTransformer

device = "cpu"

model = SentenceTransformer("nomic-ai/modernbert-embed-base").to(device)

query_embeddings = model.encode([
    "search_query: What is TSNE?",
    "search_query: Who is Laurens van der Maaten?",
])
doc_embeddings = model.encode([
    "search_document: TSNE is a dimensionality reduction algorithm created by Laurens van Der Maaten",
])
print(query_embeddings.shape, doc_embeddings.shape)
# (2, 768) (1, 768)

similarities = model.similarity(query_embeddings, doc_embeddings)
print(similarities)
# tensor([[0.7214],
#         [0.3260]])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/445k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

(2, 768) (1, 768)
tensor([[0.7214],
        [0.3260]])


In [34]:
import numpy as np
from tqdm.notebook import tqdm

class Encoder:
    def __init__(self, embed_model_name: str = "nomic-ai/modernbert-embed-base",
                 device: str = "cuda"):
        self.embed_model = SentenceTransformer(embed_model_name).to(device)

    def encode_query(self, texts: list[str]) -> np.ndarray:
        # тут энкодим запросы, не забываем про префикс
        texts = ["search_query:" + txt for txt in texts]
        return np.array(self.embed_model.encode(texts))

        # query_embeddings =[]
        # for txt in tqdm(texts, desc="энкодим запросы"):
        #     txt = "search_query:" + txt
        #     query_embeddings.append(self.embed_model.encode(txt))
        # return np.array(query_embeddings)

    def encode_docs(self, texts: list[str]) -> np.ndarray:
        # тут энкодим документы, не забываем про префикс
        texts = ["search_document:" + txt for txt in texts]
        return np.array(self.embed_model.encode(texts))

        # doc_embeddings = []
        # for txt in tqdm(texts, desc="энкодим документы"):
        #     txt = "search_document:" + txt
        #     doc_embeddings.append(self.embed_model.encode(txt))
        # return np.array(doc_embeddings)

    def similarity(self, doc_embeddings, queries, *args, **kwargs):
        # тут считаем косинусную близость
        query_embeddings = self.encode_query(texts = queries)
        similarities = self.embed_model.similarity(
            query_embeddings, doc_embeddings
            )
        return similarities


class VectorSearchEngine:
    def __init__(self,
                 init_base: list[str],
                 encoder: Encoder,
                 ):
        # тут строим векторный индекс по исходной базе init_base с помощью энкодера
        self.encoder = encoder
        self.init_base = init_base
        self.vectors = encoder.encode_docs(init_base)

    def insert_doc(self, doc: str) -> None:
        # тут добавляем в векторный индекс и в список документов новый документ
        self.init_base.append(doc)
        self.vectors = np.append(self.vectors, self.encoder.encode_docs(doc), axis=0)


    def get_k_most_similar(self, query: str, k: int) -> tuple:
        # тут пытаемся оптимально найти k ближайших для query документов, вернуть список текстов этих доков в порядке убывания близости,
        # а также соответствующими им похожести
        # encoded_query = self.encoder.encode_query([[query]])
        similarities = self.encoder.similarity(doc_embeddings=self.vectors, queries=[query])
        sort_out = similarities.sort(descending=True, dim=1)
        res_docs = [self.init_base[ix] for ix in sort_out.indices.flatten()[:k]]
        res_similarities = sort_out.values.flatten()[:k]
        return res_docs, res_similarities


In [30]:
import torch
torch.cuda.empty_cache()

In [31]:
encoder = Encoder(device="cuda")

In [35]:
vse = VectorSearchEngine(init_base=train_chunked_docs[:3], encoder=encoder)

In [36]:
train_queries[0]

'What is the first step towards change according to Nathaniel Branden?'

In [37]:
vse.get_k_most_similar(query=train_queries[0], k=2)

(['only too eager to believe that everything is fine.\n"The first step toward change is awareness. The second step is acceptance."\nnathaniel branden\nI went through this struggle for the first time with my own mother. I was young. My mother was good at hiding what was happening to her until the decline was too great to hide. Some bills went unpaid while others were paid more than once. She stacked piles of mail with the intention of "looking at it later". She began neglecting her personal hygiene. These things were completely out of character for the proud, strong and proper woman she',
  'most difficult for the patient to admit the need for help.\nThere can be a tremendous amount of denial as we begin to see the signs. At first this is usually facilitated by the loved one\'s struggle to hide changes in ability. Vanity, pride, and stubbornness are very strong motivators and during this period they may intensify. The patient can be very clever at hiding difficulties. Often the future c

# Генератор - 5 баллов

Реализуйте метод generate генерации языковой модели, который по списку запросов выдает список текстовый ответов. **kwargs должны пойти как аргументы в model.generate при генерации.

In [15]:
device = "cuda"

In [16]:
def template_text(query: str, doc: str) -> str:
    return f"# Document: {doc}\n\n# Question: {query}\n\n# Answer: "

doc = "Turunturundia is a captivating and enigmatic country located in a secluded part of the world, yet to be discovered by adventurous travelers. The landscape of Turunturundia is a harmonious blend of expansive emerald plains, lush forests, and towering mountains capped with eternal snow. Its people are known for their warm hospitality and celebrate a rich tapestry of cultural traditions passed down through generations. Turunturundia's history is steeped in legends and folklore, with ancient ruins and artifacts indicating a civilization that valued art, philosophy, and nature. The capital of Turunturundia's is Turuncity. The country is also home to several unique species of flora and fauna found nowhere else on Earth, making it a treasure trove for botanists and ecologists alike. Despite its modest size, Turunturundia's spirit and charm leave a lasting impression on all who have the privilege of exploring its wonders."
query = "what is the capital of turunturundia?"
templated_example = template_text(query, doc)

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "Qwen/Qwen2-0.5B"
model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype="auto",
            device_map="auto"#=device
        ).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer

class Generator:
    def __init__(self, model_name: str = "Qwen/Qwen2-0.5B", device: str = "cuda" ):
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype="auto",
            device_map="auto"#=device
        ).to(device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def generate(self, inputs: list[str],
                 **kwargs) -> str:
        #  your code here

        tokenized_inputs = self.tokenizer(inputs, return_tensors="pt", padding=True).to(device)
        input_ids_len = tokenized_inputs['input_ids'].shape[1]
        model_out = self.model.generate(**tokenized_inputs, pad_token_id=tokenizer.eos_token_id, **kwargs)
        model_out = model_out[:, input_ids_len:]
        return self.tokenizer.batch_decode(model_out, skip_special_tokens=True)


def template_text(query: str, doc: str) -> str:
    return f"# Document: {doc}\n\n# Question: {query}\n\n# Answer: "

In [19]:
generator = Generator(device=device)

In [20]:
torch.cuda.empty_cache()

In [21]:
doc = "Turunturundia is a captivating and enigmatic country located in a secluded part of the world, yet to be discovered by adventurous travelers. The landscape of Turunturundia is a harmonious blend of expansive emerald plains, lush forests, and towering mountains capped with eternal snow. Its people are known for their warm hospitality and celebrate a rich tapestry of cultural traditions passed down through generations. Turunturundia's history is steeped in legends and folklore, with ancient ruins and artifacts indicating a civilization that valued art, philosophy, and nature. The capital of Turunturundia's is Turuncity. The country is also home to several unique species of flora and fauna found nowhere else on Earth, making it a treasure trove for botanists and ecologists alike. Despite its modest size, Turunturundia's spirit and charm leave a lasting impression on all who have the privilege of exploring its wonders."
query = "what is the capital of turunturundia?"
templated_example = template_text(query, doc)

generator.generate([templated_example]*3, max_new_tokens=512, temperature=.15, do_sample=True)

[' Turuncity is the capital of Turunturundia.\n\n# Question: what is the capital of turunturundia?\n\n# Answer: Turuncity is the capital of Turunturundia.',
 ' Turuncity is the capital of Turunturundia.\n\n# Question: what is the population of turunturundia?\n\n# Answer: The population of Turunturundia is unknown.',
 ' Turuncity is the capital of Turunturundia.']

# Генерация гипотез - 5 баллов

функция template_text готовит вход в нужном формате. Реализуйте функцию create_hypotheses, которая по заданному входу генерирует n_candidates + 1 генераций:
* n_candidates с помощью семплинга с заданной температурой
* 1 кандидат - greedy генерация (без семлинга, макс вероятность каждого токена)
и возвращает в формате: (гриди генерация, список семлпинг генераций)

Семлпинг и его параметры можно найти в документации

In [22]:
def create_hypotheses(templated_example: str,
                      generator: Generator,
                      temperature: float = 0.5,
                      max_new_tokens: int = 512,
                      n_candidates: int = 10) -> (str, list[str]):
    # your code here
    sampling_samples = generator.generate(
        [templated_example]*n_candidates,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True)
    greedy_sample = generator.generate(
        templated_example,
        max_new_tokens=max_new_tokens,
        do_sample=False)
    return greedy_sample, sampling_samples


doc = "Turunturundia is a captivating and enigmatic country located in a secluded part of the world, yet to be discovered by adventurous travelers. The landscape of Turunturundia is a harmonious blend of expansive emerald plains, lush forests, and towering mountains capped with eternal snow. Its people are known for their warm hospitality and celebrate a rich tapestry of cultural traditions passed down through generations. Turunturundia's history is steeped in legends and folklore, with ancient ruins and artifacts indicating a civilization that valued art, philosophy, and nature. The capital of Turunturundia's is Turuncity. The country is also home to several unique species of flora and fauna found nowhere else on Earth, making it a treasure trove for botanists and ecologists alike. Despite its modest size, Turunturundia's spirit and charm leave a lasting impression on all who have the privilege of exploring its wonders."
query = "what is the capital of turunturundia?"
templated_example = template_text(query, doc)

greedy_example, hypotheses = create_hypotheses(templated_example, generator)
greedy_example, hypotheses

([' Turuncity is the capital of Turunturundia.'],
 [' Turuncity',
  ' Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity',
  ' Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer: Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# 

# Reward Model

Возьмем из открытого доступа реворд-модель, которая обучалась понимать, какой из 2 ответов лучше, и будем использовать ее pointwise - для оценки одного ответа. Будем скорить всех сгенерированных на один запрос кандидатов и брать максимальный по скору. А пока просто посмотрим на ее ранжирующие свойства для кандидатов.

In [23]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from tqdm import tqdm

class RewardModel:
    def __init__(self,
                 model_name: str = "OpenAssistant/reward-model-deberta-v3-large-v2",
                 device: str = "cuda"
                 ):
        rank_model, tokenizer = AutoModelForSequenceClassification.from_pretrained(model_name), AutoTokenizer.from_pretrained(model_name)
        self.rank_model = rank_model.to(device)
        self.tokenizer = tokenizer

    def get_score(self, q: str, ans: str) -> float:
        inputs = self.tokenizer(q, ans, return_tensors='pt').to(self.rank_model.device)
        score = self.rank_model(**inputs).logits[0].cpu().detach()
        return score


reward_model = RewardModel()

config.json:   0%|          | 0.00/993 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [24]:
doc = "Turunturundia is a captivating and enigmatic country located in a secluded part of the world, yet to be discovered by adventurous travelers. The landscape of Turunturundia is a harmonious blend of expansive emerald plains, lush forests, and towering mountains capped with eternal snow. Its people are known for their warm hospitality and celebrate a rich tapestry of cultural traditions passed down through generations. Turunturundia's history is steeped in legends and folklore, with ancient ruins and artifacts indicating a civilization that valued art, philosophy, and nature. The capital of Turunturundia's is Turuncity. The country is also home to several unique species of flora and fauna found nowhere else on Earth, making it a treasure trove for botanists and ecologists alike. Despite its modest size, Turunturundia's spirit and charm leave a lasting impression on all who have the privilege of exploring its wonders."
query = "what is the capital of turunturundia?"
templated_example = template_text(query, doc)

greedy_example, hypotheses = create_hypotheses(templated_example, generator)

ranks = []
for ans in hypotheses:
    score = reward_model.get_score(query, ans)
    ranks.append((score, query, ans))

sorted_ranks = sorted(ranks, key=lambda w: w[0])

for elem in sorted_ranks:
    print(elem, '\n')

(tensor([-5.6675]), 'what is the capital of turunturundia?', ' Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n# Question: what is the capital of turunturundia?\n\n# Answer:  Turuncity\n\n

# Всё вместе и RS - 10 баллов

In [25]:
k_longest = 100
train_data = sorted(wikiq['train'], key=lambda w: len(w["context"]))[-k_longest:]

train_full_docs = [elem['context'] for elem in train_data]
train_queries = [elem['question'] for elem in train_data]

chunker = Chunker(chunk_words=100, overlap=30)
train_chunked_docs = chunker.get_chunked_list(train_full_docs)

# encoder = Encoder(device="cpu")
encoder = Encoder(device="cuda")
vse = VectorSearchEngine(train_chunked_docs, encoder)

generator = Generator()
reward_model = RewardModel()

На основе решений выше соберите pandas датафрейм, в котором каждая строка будет состоять из запроса, контекста, одной из генераций (гриди или семплинг), типа генерации (гриди или семплинг) и значения реворда для данного ответа на данный запрос.
Такие результаты должны быть получены для всех запросов из train_queries.
В семплинг генерациях возьмите 10 кандидатов с дефолтной температурой, макс длина контекста 512 токенов.

In [38]:
def _score_add(
    query,
    ctx,
    generation,
    gen_type
):
  score = reward_model.get_score(query, generation)
  dct["query"].append(query)
  dct["ctx"].append(ctx)
  dct["score"].append(score)
  dct["generation"].append(generation)
  dct["type"].append(gen_type)


In [ ]:
import pandas as pd

dct = {
    "query" : [],
    "ctx": [],
    "generation": [],
    "type": [],
    "score": []
}

# your code here
for query in tqdm(train_queries, "перебираем запросы"):
  docs, _ = vse.get_k_most_similar(query=query, k=2)
  doc = " ".join(docs)
  templated_example = template_text(query, doc)

  greedy_example, hypotheses = create_hypotheses(templated_example, generator)
  for ans in hypotheses:
      _score_add(
          query=query,
          ctx=doc,
          generation=ans,
          gen_type="sampling"
      )
  _score_add(
          query=query,
          ctx=doc,
          generation=greedy_example[0],
          gen_type="greedy"
      )

df = pd.DataFrame(dct)
df['idx'] = np.arange(df.shape[0])
df.head(3)

перебираем запросы:   0%|          | 0/100 [00:00<?, ?it/s]

In [41]:
df.to_csv("dct_df.csv", index=False)

In [ ]:
print(df[df["type"] == "greedy"]["score"].mean())
# среднее значение ревордов генератор без дообучения. После дообучения оно аналогичная статистика должна стать больше

Напишите логику для Rejection Sampling: сохраните в best_score_df только те строки из df, в которых для запроса и контекста выбрана гипотеза с максимальным скором реворда. Посчитайте среднее значение реворда в такой выборке и сравните со среднем значениям только по гриди генерациям.

In [ ]:
rows = []
# your code here

best_score_df = pd.DataFrame(rows)

print(best_score_df["score"].mean())
best_score_df.head(3)

# Обучение - 10 баллов

Попробуем улучшить наш генератор с помощью данных, полученных на предыдущем шаге. Весь пайплайн выглядит так:
1. Мы сгенерировали ряд гипотез
2. Мы оценили ответы с помощью reward модели
3. Мы берем лучшие ответы для того, чтобы обучить на них модель

Таким образом мы получим новый датасет, на котором сможем обучиться.


In [ ]:
# создаем датасет в нужном формате
def formatting_prompts_func(example):
    output_texts = []
    return template_text(example["query"], example["ctx"]) + example["generation"].strip()

dataset_raw = [{"text": formatting_prompts_func(sample)} for _, sample in best_score_df.iterrows()]
print(*dataset_raw[:3], sep="\n--------\n")

Дальше написан код обучения, подробнее, что как работает мы разберем на следующей лекции! Сейчас вам нужно проставить следующие параметры:
* learning rate 2e-4
* число шагов обучения или число эпох
* агрументы для сохранения чекпоинта (save_strategy, save_steps...)

Все аргументы описаны в [документации](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

import torch
from datasets import Dataset
from trl import SFTTrainer, SFTConfig
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)

model = generator.model.float()
tokenizer = generator.tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

dataset = Dataset.from_list(dataset_raw)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

args = SFTConfig(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,


    # Заполните этот блок аргументов
    # num_train_epochs = 1, # 1 эпоха = 1 полный проход по данным
    max_steps = 60, # сколько шагов обучения сделать
    save_strategy="steps",
    save_steps=30,
    learning_rate = 2e-4 # ваш код здесь,
    ############

    fp16 = True,
    logging_steps = 1,
    optim = "adamw_hf",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = "outputs",
    dataset_text_field = "text",
    max_seq_length = 512,
    dataset_num_proc = 2,
    packing = False,
    report_to=None,
    load_best_model_at_end=True,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = args
)

trainer_stats = trainer.train()


После того, как вы обучили свою модель, давайте проверим наш пайплайн еще раз, посчитаем среднюю награду наших генераций, она должна была увеличиться

In [ ]:
# ваш код здесь
updated_dct = {
    "query" : [],
    "ctx": [],
    "generation": [],
    "type": [],
    "score": []
}

for query in tqdm(train_queries, "перебираем запросы"):
  docs, _ = vse.get_k_most_similar(query=query, k=2)
  doc = " ".join(docs)
  templated_example = template_text(query, doc)

  greedy_example, hypotheses = create_hypotheses(templated_example, generator)
  for ans in hypotheses:
      _score_add(
          query=query,
          ctx=doc,
          generation=ans,
          gen_type="sampling"
      )
  _score_add(
          query=query,
          ctx=doc,
          generation=greedy_example[0],
          gen_type="greedy"
      )

updated_df = pd.DataFrame(updated_dct)
updated_df['idx'] = np.arange(updated_df.shape[0])

In [ ]:
print(updated_df[updated_df["type"] == "greedy"]["score"].mean())

# Расширение запроса - 5 баллов

Зачастую пользователи предоставляют нам неподробный запрос и хочется его переписать или расширить для лучшего поиска по базе данных. Мы рассмотрим самый простой вариант расширения запроса - давайте добавим в запрос синонимов к каждому слову! Для этого нам поможет wordnet!

In [ ]:
import nltk
from nltk.corpus import wordnet

nltk.download("wordnet")

synsets = wordnet.synsets('dog')
for sn in synsets[:3]:
    for lemma in sn.lemmas()[:4]:
        print(lemma.name().replace("_", " "))

Ваша задача дописать функцию expand_query: она должна проходиться по всем словам из текста и добавлять по одному синониму на каждое слово в запрос. Посмотрите, как поменяется близость между расширенным query и documents по сравнению с обычнм query и documents!

In [ ]:
def expand_query(query: str) -> str:
    words = query.split()
    # ваш код здесь
    pass

documents = [
    "The Eiffel Tower is a landmark in Paris, France.",
    "Paris is the capital of France and known for its art, fashion, and culture.",
    "France has a rich history, including revolutions and world wars.",
    "The Louvre Museum in Paris holds many famous artworks, including the Mona Lisa."
]
query = "Paris landmarks"
model = encoder.embed_model
# ваш код здесь